# Advanced Indexing — Practice (with Solutions)

This notebook contains **8 advanced** indexing problems covering:
- Python slicing & negative steps
- NumPy multi-dimensional indexing
- Slicing vs fancy indexing (**views vs copies**)
- Boolean masking
- `np.ix_` for row/column selection
- Safe assignment with fixed-width dtypes (avoiding wraparound)
- `...` (ellipsis) and `None`/`np.newaxis`

**Best practice:** Try each exercise cell before opening the solution cell below it.

In [1]:
import numpy as np

np.set_printoptions(edgeitems=10, linewidth=120)
RNG = np.random.default_rng(0)

## Exercise 1 — Center cross of an odd square matrix

Given a **square** NumPy array `mat` with **odd** size `n x n`, write a function:

```python
def center_cross(mat):
    ...
```

that returns a **tuple** `(row, col)` where:
- `row` is the middle row (1D view)
- `col` is the middle column (1D view)

Constraints:
- Use indexing/slicing only (no loops).
- The outputs should be 1D arrays.


In [2]:
# Try it
mat1 = np.arange(1, 26).reshape(5, 5)
mat1

array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25]])

In [3]:
# YOUR CODE HERE
def center_cross(mat: np.ndarray):
    raise NotImplementedError

# row, col = center_cross(mat1)
# row, col

### Solution 1

In [4]:
def center_cross(mat: np.ndarray):
    if mat.ndim != 2 or mat.shape[0] != mat.shape[1]:
        raise ValueError("mat must be a square 2D array")
    n = mat.shape[0]
    if n % 2 == 0:
        raise ValueError("n must be odd")
    mid = n // 2
    row = mat[mid, :]      # 1D view
    col = mat[:, mid]      # 1D view
    return row, col

row, col = center_cross(mat1)
print("mat:\n", mat1)
print("middle row:", row)
print("middle col:", col)

assert row.shape == (5,) and col.shape == (5,)
assert np.array_equal(row, np.array([11, 12, 13, 14, 15]))
assert np.array_equal(col, np.array([3, 8, 13, 18, 23]))

mat:
 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]]
middle row: [11 12 13 14 15]
middle col: [ 3  8 13 18 23]


## Exercise 2 — Replace the border of a 2D array (in-place)

Given a 2D NumPy array `a`, set its **border elements** (top row, bottom row, left column, right column) to a value `v`.

Write:
```python
def set_border(a, v):
    ...
```

Requirements:
- Must modify `a` **in-place**.
- Use slicing/indexing only.
- No loops.


In [5]:
# Try it
a2 = np.arange(1, 1 + 6*8).reshape(6, 8)
a2

array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29, 30, 31, 32],
       [33, 34, 35, 36, 37, 38, 39, 40],
       [41, 42, 43, 44, 45, 46, 47, 48]])

In [6]:
# YOUR CODE HERE
def set_border(a: np.ndarray, v):
    raise NotImplementedError

# set_border(a2, -1)
# a2

### Solution 2

In [7]:
def set_border(a: np.ndarray, v):
    if a.ndim != 2:
        raise ValueError("a must be 2D")
    a[0, :] = v
    a[-1, :] = v
    a[:, 0] = v
    a[:, -1] = v

a2 = np.arange(1, 1 + 6*8).reshape(6, 8)
set_border(a2, -1)
print(a2)

assert np.all(a2[0, :] == -1)
assert np.all(a2[-1, :] == -1)
assert np.all(a2[:, 0] == -1)
assert np.all(a2[:, -1] == -1)
assert a2[1, 1] != -1

[[-1 -1 -1 -1 -1 -1 -1 -1]
 [-1 10 11 12 13 14 15 -1]
 [-1 18 19 20 21 22 23 -1]
 [-1 26 27 28 29 30 31 -1]
 [-1 34 35 36 37 38 39 -1]
 [-1 -1 -1 -1 -1 -1 -1 -1]]


## Exercise 3 — Python slicing with negative step (advanced but common)

Given a Python list `xs`, create a new list containing:
- every **2nd** element of `xs`, starting from the end, i.e. indices `-1, -3, -5, ...`

Do it using **one slice** (no loops).

In [8]:
xs = list("ABCDEFGHIJKL")
xs

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']

In [9]:
# YOUR CODE HERE
# result3 = ...
# result3

### Solution 3

In [10]:
result3 = xs[::-2]
print(result3)
assert result3 == ['L', 'J', 'H', 'F', 'D', 'B']

['L', 'J', 'H', 'F', 'D', 'B']


## Exercise 4 — NumPy: slicing vs fancy indexing (view vs copy)

This exercise checks whether you understand when NumPy returns a **view** vs a **copy**.

Given `a` below:
- Create `s1` as the subarray consisting of rows `1:4` and columns `2:5` using **slicing**.
- Create `s2` as the same region but using **fancy indexing** with explicit row/col index arrays.

Then:
- Modify `s1[0,0] = -999` and observe whether `a` changed.
- Modify `s2[0,0] = -555` and observe whether `a` changed.

Goal: choose indexing so that **one** of these changes affects `a` and the other does not.

In [11]:
a4 = np.arange(1, 1 + 6*7).reshape(6, 7)
a4

array([[ 1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19, 20, 21],
       [22, 23, 24, 25, 26, 27, 28],
       [29, 30, 31, 32, 33, 34, 35],
       [36, 37, 38, 39, 40, 41, 42]])

In [12]:
# YOUR CODE HERE
# s1 = ...
# s2 = ...
#
# a4_before = a4.copy()
# s1[0,0] = -999
# s2[0,0] = -555
#
# print("a changed by s1?", not np.array_equal(a4, a4_before))
# print("a now:\n", a4)
# print("s1:\n", s1)
# print("s2:\n", s2)

### Solution 4

In [13]:
a4 = np.arange(1, 1 + 6*7).reshape(6, 7)

# Slicing -> view (typically)
s1 = a4[1:4, 2:5]

# Fancy indexing -> copy
rows = np.array([1, 2, 3])
cols = np.array([2, 3, 4])
s2 = a4[np.ix_(rows, cols)]

a4_before = a4.copy()
s1[0, 0] = -999
changed_after_s1 = not np.array_equal(a4, a4_before)

a4_before2 = a4.copy()
s2[0, 0] = -555
changed_after_s2 = not np.array_equal(a4, a4_before2)

print("a changed by s1 (slice/view)?", changed_after_s1)
print("a changed by s2 (fancy/copy)?", changed_after_s2)
print("a4:\n", a4)

assert changed_after_s1 is True
assert changed_after_s2 is False
assert a4[1, 2] == -999

a changed by s1 (slice/view)? True
a changed by s2 (fancy/copy)? False
a4:
 [[   1    2    3    4    5    6    7]
 [   8    9 -999   11   12   13   14]
 [  15   16   17   18   19   20   21]
 [  22   23   24   25   26   27   28]
 [  29   30   31   32   33   34   35]
 [  36   37   38   39   40   41   42]]


## Exercise 5 — Boolean masking without modifying the original

You have a 1D array `x`. Create a new array `y` such that:
- values `< threshold` become `0`
- values `>= threshold` stay the same

Constraints:
- Do **not** modify `x`.
- Use boolean indexing.


In [14]:
x5 = np.array([3, 10, 2, 8, 8, 1, 12])
threshold = 8
x5

array([ 3, 10,  2,  8,  8,  1, 12])

In [15]:
# YOUR CODE HERE
# y5 = ...
# print("x5:", x5)
# print("y5:", y5)

### Solution 5

In [16]:
x5 = np.array([3, 10, 2, 8, 8, 1, 12])
threshold = 8

y5 = x5.copy()
y5[y5 < threshold] = 0

print("x5:", x5)
print("y5:", y5)

assert np.array_equal(x5, np.array([3, 10, 2, 8, 8, 1, 12]))  # unchanged
assert np.array_equal(y5, np.array([0, 10, 0, 8, 8, 0, 12]))

x5: [ 3 10  2  8  8  1 12]
y5: [ 0 10  0  8  8  0 12]


## Exercise 6 — Select a submatrix by rows and columns using `np.ix_`

Given a 2D array `A`, and two Python lists:
- `row_ids` (rows to keep)
- `col_ids` (columns to keep)

Create `B` which is the submatrix formed by taking **all combinations** of those rows and columns.

Example: if `row_ids=[0,2]` and `col_ids=[1,3]`, then `B` is shape `(2,2)` with entries:
- rows 0 and 2
- cols 1 and 3

Use indexing only (no loops).

In [17]:
A6 = np.arange(1, 1 + 5*6).reshape(5, 6)
row_ids = [0, 2, 4]
col_ids = [1, 3, 5]
A6

array([[ 1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12],
       [13, 14, 15, 16, 17, 18],
       [19, 20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29, 30]])

In [18]:
# YOUR CODE HERE
# B6 = ...
# B6

### Solution 6

In [19]:
A6 = np.arange(1, 1 + 5*6).reshape(5, 6)
row_ids = [0, 2, 4]
col_ids = [1, 3, 5]

B6 = A6[np.ix_(row_ids, col_ids)]
print(B6)

expected6 = np.array([
    [ 2,  4,  6],
    [14, 16, 18],
    [26, 28, 30]
])
assert np.array_equal(B6, expected6)
assert B6.shape == (3, 3)

[[ 2  4  6]
 [14 16 18]
 [26 28 30]]


## Exercise 7 — Safe assignment into fixed-width integer arrays

NumPy integer arrays with fixed-width dtypes (like `uint8`) can **wrap around** when you assign values outside the valid range.

Write a function:
```python
def safe_assign(arr, idx, value):
    ...
```

that assigns into `arr[idx]` but **clips** `value` into the dtype's valid range.

Requirements:
- Must work for integer dtypes (signed/unsigned).
- Use `np.iinfo(arr.dtype)`.
- Should modify `arr` in-place.


In [20]:
arr7 = np.array([10, 20, 30, 40], dtype=np.uint8)
arr7

array([10, 20, 30, 40], dtype=uint8)

In [21]:
# YOUR CODE HERE
# def safe_assign(arr, idx, value):
#     ...
#
# safe_assign(arr7, 0, -100)
# safe_assign(arr7, 1, 999)
# arr7

### Solution 7

In [22]:
def safe_assign(arr: np.ndarray, idx, value):
    """Assign value into arr[idx], clipping to the dtype range (integer dtypes only)."""
    if not np.issubdtype(arr.dtype, np.integer):
        raise TypeError("safe_assign only supports integer dtypes")
    info = np.iinfo(arr.dtype)
    arr[idx] = int(np.clip(value, info.min, info.max))


arr7 = np.array([10, 20, 30, 40], dtype=np.uint8)

# Wrap demonstration that avoids passing a negative Python int directly to uint8:
wrapped_value = np.array(-100, dtype=np.int16).astype(np.uint8)[()]  # scalar uint8
print("wrapped (mod 256):", int(wrapped_value))  # 156

# Safe assignment clips instead of wrapping
safe_assign(arr7, 0, -100)  # -> 0
safe_assign(arr7, 1, 999)   # -> 255

print("safe:", arr7)

assert int(wrapped_value) == 156
assert arr7[0] == 0
assert arr7[1] == 255


wrapped (mod 256): 156
safe: [  0 255  30  40]


## Exercise 8 — Ellipsis (`...`) and `None` / `np.newaxis`

Given a 3D array `T` with shape `(2, 3, 4)`:

1) Extract `last_col` which contains the **last element along the last axis** for every `(i, j)` pair.
   - The result should have shape `(2, 3)`.
   - Use ellipsis.

2) Create `last_col_3d` from `last_col` but with an extra trailing dimension so it has shape `(2, 3, 1)`.
   - Use `None` or `np.newaxis`.


In [23]:
T8 = np.arange(2*3*4).reshape(2, 3, 4)
T8, T8.shape

(array([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],
 
        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]]),
 (2, 3, 4))

In [24]:
# YOUR CODE HERE
# last_col = ...
# last_col_3d = ...
#
# print(last_col)
# print(last_col.shape)
# print(last_col_3d.shape)

### Solution 8

In [25]:
T8 = np.arange(2*3*4).reshape(2, 3, 4)

last_col = T8[..., -1]         # shape (2, 3)
last_col_3d = last_col[..., None]  # shape (2, 3, 1)

print("T8 shape:", T8.shape)
print("last_col:\n", last_col)
print("last_col shape:", last_col.shape)
print("last_col_3d shape:", last_col_3d.shape)

assert last_col.shape == (2, 3)
assert last_col_3d.shape == (2, 3, 1)
assert np.array_equal(last_col, np.array([[3, 7, 11], [15, 19, 23]]))

T8 shape: (2, 3, 4)
last_col:
 [[ 3  7 11]
 [15 19 23]]
last_col shape: (2, 3)
last_col_3d shape: (2, 3, 1)


## Quick recap

- **Slicing** (e.g. `a[1:4, 2:5]`) usually returns a **view** (changes can affect the original).
- **Fancy indexing** (e.g. `a[[1,2,3], [2,3,4]]` or `a[np.ix_(rows, cols)]`) returns a **copy**.
- **Boolean masks** are powerful for selection and in-place updates.
- Fixed-width integer dtypes can **wrap**; clip if you want safety.
- Use `...` and `None` to write clear indexing for higher-dimensional arrays.
